In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, '../../../')

from notebooks.utils import _ALEXA_DATA_PATH, load_node_features, load_level_data, create_nodes, export_model_as_feature
from train import run_experiment

2022-01-09 16:04:16.524335: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load audience overlap edges for level 1

In [2]:
audience_overlap_sites = load_level_data(os.path.join(_ALEXA_DATA_PATH, 'corpus_2020_audience_overlap_sites_scrapping_result.json'), level=1)
audience_overlap_sites_NODES = create_nodes(audience_overlap_sites)

print(audience_overlap_sites_NODES[:5])

01-09 16:04:18 notebooks.utils INFO     Loaded 3489 nodes with records level <= 1 and child size:16981


[('crooked.com', 'votesaveamerica.com'), ('crooked.com', 'art19.com'), ('crooked.com', 'promocodeportal.com'), ('crooked.com', 'mediamatters.org'), ('crooked.com', 'actblue.com')]


In [3]:
edge_df = pd.DataFrame(audience_overlap_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,crooked.com,votesaveamerica.com
1,crooked.com,art19.com
2,crooked.com,promocodeportal.com
3,crooked.com,mediamatters.org
4,crooked.com,actblue.com


### 1. Load all node features

In [4]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


In [5]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104905 entries, whistleblowersandrelators.com to usopenfinals.de
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   63788 non-null  float64
 1   daily_pageviews_per_visitor  63803 non-null  float64
 2   daily_time_on_site           38024 non-null  float64
 3   total_sites_linking_in       99398 non-null  float64
 4   bounce_rate                  32718 non-null  float64
dtypes: float64(5)
memory usage: 4.8+ MB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [6]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104905 entries, whistleblowersandrelators.com to usopenfinals.de
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   alexa_rank                   104905 non-null  float64
 1   daily_pageviews_per_visitor  63803 non-null   float64
 2   daily_time_on_site           38024 non-null   float64
 3   total_sites_linking_in       104905 non-null  float64
 4   bounce_rate                  32718 non-null   float64
dtypes: float64(5)
memory usage: 4.8+ MB


### 3. Normalizing features

In [7]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

### Find all unique nodes in edges

In [8]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 10161 Sample: ['englishprof-playinthedirt.blogspot.com', 'marripedia.org', 'swans.com', 'inweekly.net', 'srjnews.com']


# Create Graph

In [9]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 10161, Edges: 17010

 Node types:
  default: [10161]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [17010]
        Weights: all 1 (default)
        Features: none


# Unsupervised Attrib2Vec

In [10]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 50
epochs = 4
num_samples = [10, 5]

generator = Attri2VecLinkGenerator(G, batch_size)
train_gen = generator.flow(unsupervised_samples)

layer_sizes = [128]
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)

# Build the model and expose input and output sockets of attri2vec, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

prediction = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)

model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [11]:
history = model.fit(train_gen, epochs=4, verbose=2, use_multiprocessing=False, workers=1, shuffle=True)

# """
# previous before normalization

# WARNING:tensorflow:sample_weight modes were coerced from
#   ...
#     to
#   ['...']
# Train for 1899 steps
# Epoch 1/8
# 1899/1899 - 47s - loss: 0.7380 - binary_accuracy: 0.5427
# Epoch 2/8
# 1899/1899 - 46s - loss: 0.6368 - binary_accuracy: 0.6424
# Epoch 3/8
# 1899/1899 - 47s - loss: 0.5929 - binary_accuracy: 0.6680
# Epoch 4/8
# 1899/1899 - 48s - loss: 0.5694 - binary_accuracy: 0.6800
# Epoch 5/8
# 1899/1899 - 52s - loss: 0.5564 - binary_accuracy: 0.6865
# Epoch 6/8
# 1899/1899 - 47s - loss: 0.5442 - binary_accuracy: 0.6933
# Epoch 7/8
# 1899/1899 - 48s - loss: 0.5399 - binary_accuracy: 0.6941
# Epoch 8/8
# 1899/1899 - 47s - loss: 0.5272 - binary_accuracy: 0.7013

# """

Epoch 1/4
1626/1626 - 16s - loss: 0.6971 - binary_accuracy: 0.5276 - 16s/epoch - 10ms/step
Epoch 2/4
1626/1626 - 16s - loss: 0.6697 - binary_accuracy: 0.5415 - 16s/epoch - 10ms/step
Epoch 3/4
1626/1626 - 15s - loss: 0.6632 - binary_accuracy: 0.5396 - 15s/epoch - 9ms/step
Epoch 4/4
1626/1626 - 16s - loss: 0.6608 - binary_accuracy: 0.5357 - 16s/epoch - 10ms/step


In [13]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

node_gen = Attri2VecNodeGenerator(G, batch_size).flow(nodes_in_edges)
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=1)

embeddings_wv = dict(zip(nodes_in_edges, node_embeddings.tolist()))

print('Sample:', embeddings_wv['crooked.com'][:10])

204/204 [==============================] - 0s 756us/step
Sample: [0.03328922390937805, 0.04732587933540344, 0.0591701865196228, 0.02707517147064209, 0.02760675549507141, 0.024059206247329712, 0.043164610862731934, 0.043186068534851074, 0.03645527362823486, 0.04844653606414795]


# Export embeddings as feature

In [14]:
export_model_as_feature(embeddings_wv, 'attrib2vec')

'/Users/panayot/Documents/News-Media-Peers/data/acl2020/features/attrib2vec.json'

In [15]:
run_experiment(features="attrib2vec")

+------+---------------------+---------------+--------------------+------------+
| task | classification_mode | type_training | normalize_features |  features  |
+------+---------------------+---------------+--------------------+------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec |
+------+---------------------+---------------+--------------------+------------+


01-09 16:07:01 train        INFO     Start training...
01-09 16:07:01 train        INFO     Fold: 0
01-09 16:07:06 train        INFO     Fold: 1
01-09 16:07:10 train        INFO     Fold: 2
01-09 16:07:13 train        INFO     Fold: 3
01-09 16:07:15 train        INFO     Fold: 4


+------+---------------------+---------------+--------------------+------------+-------------------+--------------------+--------------------+--------------------+
| task | classification_mode | type_training | normalize_features |  features  |      Macro-F1     |      Accuracy      |  Flip error-rate   |        MAE         |
+------+---------------------+---------------+--------------------+------------+-------------------+--------------------+--------------------+--------------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec | 34.12428926388317 | 53.783469150174625 | 15.250291036088475 | 0.6146682188591386 |
+------+---------------------+---------------+--------------------+------------+-------------------+--------------------+--------------------+--------------------+
